In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
from numpy import dstack
from pandas import read_csv
 
# load a single file as a numpy array
def load_file(filepath):
 dataframe = read_csv(filepath, header=None, delim_whitespace=True)
 return dataframe.values
 
# load a list of files, such as x, y, z data for a given variable
def load_group(filenames, prefix=''):
 loaded = list()
 for name in filenames:
  data = load_file(prefix + name)
  loaded.append(data)
 # stack group so that features are the 3rd dimension
 loaded = dstack(loaded)
 return loaded
 
# load a dataset group, such as train or test
def load_dataset(group, prefix=''):
 filepath = prefix + group + '/Inertial Signals/'
 # load all 9 files as a single array
 filenames = list()
 # total acceleration
 filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
 # body acceleration
 filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
 # body gyroscope
 filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
 # load input data
 X = load_group(filenames, filepath)
 # load class output
 y = load_file(prefix + group + '/y_'+group+'.txt')
 return X, y
 
# load all train
trainX, trainy = load_dataset('train', '/content/drive/My Drive/datasets/UCI HAR Dataset/')
print(trainX.shape, trainy.shape)
# load all test
testX, testy = load_dataset('test', '/content/drive/My Drive/datasets/UCI HAR Dataset/')
print(testX.shape, testy.shape)

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)


In [3]:
import pandas as pd
trainX

array([[[ 1.012817e+00, -1.232167e-01,  1.029341e-01, ...,
          3.019122e-02,  6.601362e-02,  2.285864e-02],
        [ 1.022833e+00, -1.268756e-01,  1.056872e-01, ...,
          4.371071e-02,  4.269897e-02,  1.031572e-02],
        [ 1.022028e+00, -1.240037e-01,  1.021025e-01, ...,
          3.568780e-02,  7.485018e-02,  1.324969e-02],
        ...,
        [ 1.018445e+00, -1.240696e-01,  1.003852e-01, ...,
          3.985177e-02,  1.909445e-03, -2.170124e-03],
        [ 1.019372e+00, -1.227451e-01,  9.987355e-02, ...,
          3.744932e-02, -7.982483e-05, -5.642633e-03],
        [ 1.021171e+00, -1.213260e-01,  9.498741e-02, ...,
          2.881781e-02, -3.771800e-05, -1.446006e-03]],

       [[ 1.018851e+00, -1.239760e-01,  9.792958e-02, ...,
          1.711106e-02,  6.122797e-03,  1.226815e-02],
        [ 1.022380e+00, -1.268078e-01,  9.935086e-02, ...,
          2.417851e-02,  9.710357e-03,  1.614958e-02],
        [ 1.020781e+00, -1.277862e-01,  9.811381e-02, ...,
          3.02

In [4]:
import tensorflow as tf
trainy = trainy - 1
testy = testy - 1
trainy = tf.keras.utils.to_categorical(trainy)
testy = tf.keras.utils.to_categorical(testy)

In [5]:
from keras.optimizers import Adam

model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, 3, activation='relu', padding='same', input_shape=(128, 9)),
    tf.keras.layers.MaxPooling1D(2, padding='same'),
    tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(2, padding='same'),
   
    tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.UpSampling1D(2),
    tf.keras.layers.Conv1D(64, 3, activation='relu', padding='same'),
    tf.keras.layers.UpSampling1D(2),
    
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Reshape((64, -1)),
    tf.keras.layers.LSTM(64, return_sequences=False),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])


model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 128, 64)           1792      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 64, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 64, 32)            6176      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 32, 32)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 32, 32)            3104      
                                                                 
 up_sampling1d (UpSampling1D  (None, 64, 32)           0

In [6]:
import time
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_accuracy', mode='max', patience=3, verbose=1)

start_time = time.time()
model.fit(
  x=trainX, 
  y=trainy, 
  epochs=20,
  validation_data = (testX,testy),
  callbacks = [early_stop]
)
end_time = time.time()

Epoch 1/20
230/230 [==============================] - 26s 97ms/step - loss: 0.7340 - accuracy: 0.6668 - val_loss: 0.5806 - val_accuracy: 0.7618
Epoch 2/20
230/230 [==============================] - 21s 90ms/step - loss: 0.2814 - accuracy: 0.8921 - val_loss: 0.3942 - val_accuracy: 0.8711
Epoch 3/20
230/230 [==============================] - 21s 92ms/step - loss: 0.1431 - accuracy: 0.9427 - val_loss: 0.3633 - val_accuracy: 0.8826
Epoch 4/20
230/230 [==============================] - 20s 86ms/step - loss: 0.1327 - accuracy: 0.9416 - val_loss: 0.4279 - val_accuracy: 0.8751
Epoch 5/20
230/230 [==============================] - 24s 104ms/step - loss: 0.1800 - accuracy: 0.9335 - val_loss: 0.4012 - val_accuracy: 0.8799
Epoch 6/20
230/230 [==============================] - 20s 89ms/step - loss: 0.1250 - accuracy: 0.9489 - val_loss: 0.3660 - val_accuracy: 0.9033
Epoch 7/20
230/230 [==============================] - 21s 93ms/step - loss: 0.2463 - accuracy: 0.9143 - val_loss: 0.3876 - val_accuracy

In [7]:
print(start_time)
print(end_time)
print(end_time-start_time)

1681894136.5476992
1681894414.669917
278.12221789360046


In [8]:
start_time_testing = time.time()
test_loss, test_acc = model.evaluate(testX, testy)
end_time_testing = time.time()

93/93 [==============================] - 2s 19ms/step - loss: 0.2434 - accuracy: 0.9226


In [9]:
print(end_time_testing - start_time_testing)

2.6419360637664795
